![](https://raw.githubusercontent.com/wilocw/co2114-codebase/2024/static/0/uol_banner_red.png)

# CO2114<br />Foundations in Artificial Intelligence

# Tutorial 3 - Shortest Path


In this tutorial, we will be looking at **Dijkstra's method** for finding the shortest path between two nodes.

Dijkstra's method, also sometimes known as uniform cost search, is an example of a best-first search where the heuristic is the next closest node from the initial state. It is typically applied on weighted graphs, where each action (i.e. traversal from one node to another) has a fixed cost.

## Setup
> **Run the following cell only if you are using Google Colab**

In [ ]:
!pip install "git+https://github.com/wilocw/co2114-codebase.git@2024#egg=agent&subdirectory=lab/src/2"
!pip install "git+https://github.com/wilocw/co2114-codebase.git@2024#egg=search&subdirectory=lab/src/3"

### Imports

In [ ]:
from search.things import *
from search.graph import *

## A Weighted Graph

Consider the following graph, defined by the vertices (aka nodes), edges, and weights.

In [ ]:
graph = {
    "vertices": ["A","B","C","D","E","F"],
    "edges": [
        ["A","B"], ["A","D"], ["B","D"],
        ["B","E"], ["D","E"], ["D","F"],
        ["E","F"], ["E","C"], ["F","C"]],
    "weights": [2, 8, 5, 6, 3, 2, 1, 9, 3]
}
environment = GraphEnvironment.from_dict(graph)
environment.show(init="A")

Let's say we are interested in identifying which path between $A$ and $C$ is shortest.

### Dijkstra's Method

Dijkstra's method involves visiting nodes in order of shortest path. At each step, the next node (representing the "best" choice) is the one that has the shortest distance from the initial state. This might be a node that is not adjacent to the current state.

A record of distances is maintained as a global state of the agent. When a node is visited, its unvisited neighbours are updated such if a the distance to the neighbour is less than the distance of the current node plus the weight of the edge connecting the two nodes (i.e. the cost to travel there).

This can be written as follows. Consider that an agent is at a node, $A$ which has a neighbour, $B$. The distance to $B$ (from the initial state) is updated based on the following expression:
$d(B) = \min(d(B), d(A) + c(A,B))$.

If the distance is updated, a record of the current node is added to neighbour as it's "previous node". This indicates, e.g.,  that the shortest path to $B$ is via $A$.

In other words, 

```python
if d(B) > (d(A) + c(A,B)):
    d(B) = d(A) + c(A,B)
    p(B) = A
```

Once all the neighbours of a node have had their distances updated, the current node is marked as visited, and the agent moves to the node with the minimum distance that has not been visited. The process is updated and iterated until the target node has been reached.

Pseudo-code for the algorithm for a given `graph`, and `init` and `target` nodes is shown:

```python
def dijkstra(graph, init, target=None):
    for v in graph.nodes:
        dist[v] = infinity
        prev[v] = None
        unvisited.add(v)
    dist[init] = 0

    while target in unvisited or (target is None and unvisited not empty):
        u = None
        for v in unvisited:
            if u is None or dist[v] < dist[u]:
                u = v
    
        for v in u.neighbours:
            if v not in unvisited:
                continue
            alt = dist[u] + weight(u, v)
            if alt < dist[v]:
                dist[v] = alt
                prev[v] = u

        unvisited.remove(u)
    return dist, prev
```

Note that the distance of all nodes, `dist[v]` for `v` in `graph.nodes`, is initialised as $\infinity$: this means that any new distance is always going to be less the first time it is queried. Likewise, all nodes are added to the set of `unvisited` nodes, with the distance of the `init` node (from itself) is set to zero.

The algorthm will iterate until the `target` node is visited, or if no `target` is provided, until all nodes are visited.

The algorithm returns `dist`, the path cost between the initial node and all visited nodes, and `prev`, the "previous" node in the shortest path to each node. The latter can be used to back track to find the shortest path.

## Shortest Path from A

We will work through the algorithm in this example:

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| **A**    |  0   |     |
| _B_    | $\infty$ |     |
| C    | $\infty$ |     |
| _D_    | $\infty$ |     |
| E    | $\infty$ |     |
| F    | $\infty$ |     |

```
unvisited = [A,B,C,D,E,F]
```

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| **A**    |  0   |     |
| _B_  | 2 |  A    |
| C    | $\infty$ |     |
| _D_    | $\infty$ |     |
| E    | $\infty$ |     |
| F    | $\infty$ |     |

```
unvisited = [A,B,C,D,E,F]
```

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| **A**    |  0   |     |
| _B_    | 2 |  A    |
| C    | $\infty$ |     |
| _D_    | 8 |   A  |
| E    | $\infty$ |     |
| F    | $\infty$ |     |

```
unvisited = [B,C,D,E,F]
```

### Next node: B

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| A    |  0   |     |
| **B**    | **2** |  A    |
| C    | $\infty$ |     |
| _D_    | 8 |   A  |
| _E_    | $\infty$ |     |
| F    | $\infty$ |     |

```
unvisited = [B,C,D,E,F]
```

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| A    |  0   |     |
| **B**    | **2** |  A    |
| C    | $\infty$ |     |
| _D_    | _8_ |   A  |
| _E_    | _2+6_ |     |
| F    | $\infty$ |     |

```
unvisited = [B,C,D,E,F]
```

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| A    |  0   |     |
| **B**    | **2** |  A    |
| C    | $\infty$ |     |
| _D_   | _2+5_ |   B  |
| _E_    | _8_ |  B   |
| F    | $\infty$ |     |

```
unvisited = [B,C,D,E,F]
```

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| A    |  0   |     |
| **B**    | **2** |  A    |
| C    | $\infty$ |     |
| _D_   | _7_ |   B  |
| _E_    | _8_ |  B   |
| F    | $\infty$ |     |

```
unvisited = [C,D,E,F]
```

### Next Node: D

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| A    |  0   |     |
| B    | 2 |  A    |
| C    | $\infty$ |     |
| **D**   | **7** |   B  |
| _E_    | _8_ |  B   |
| _F_    | $\infty$ |     |

```
unvisited = [C,D,E,F]
```

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| A    |  0   |     |
| B    | 2 |  A    |
| C    | $\infty$ |     |
| **D**   | **7** |   B  |
| _E_    | _8_ |  B   |
| _F_    | _7+2_ |     |

```
unvisited = [C,D,E,F]
```

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| A    |  0   |     |
| B    | 2 |  A    |
| C    | $\infty$ |     |
| **D**   | **7** |   B  |
| _E_    | _7+3_ (> 8) |  B   |
| _F_    | _9_ |  D   |

```
unvisited = [C,D,E,F]
```

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| A    |  0   |     |
| B    | 2 |  A    |
| C    | $\infty$ |     |
| **D**   | **7** |   B  |
| _E_    | _8_ |  B   |
| _F_    | _9_ |  D   |

```
unvisited = [C,E,F]
```

### Next Node: E

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| A    |  0   |     |
| B    | 2 |  A    |
| _C_    | $\infty$ |     |
| D   | 7 |   B  |
| **E**    | **8** |  B   |
| _F_    | _9_ |  D   |

```
unvisited = [C,E,F]
```

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| A    |  0   |     |
| B    | 2 |  A    |
| _C_    | _8 + 9_ |     |
| D   | 7 |   B  |
| **E**    | **8** |  B   |
| _F_    | _9_ |  D   |

```
unvisited = [C,E,F]
```

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| A    |  0   |     |
| B    | 2 |  A    |
| _C_    | _17_ |  E   |
| D   | 7 |   B  |
| **E**    | **8** |  B   |
| _F_    | _8+1 (=9)_ |  D   |

```
unvisited = [C,E,F]
```

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| A    |  0   |     |
| B    | 2 |  A    |
| _C_    | _17_ |  E   |
| D   | 7 |   B  |
| **E**    | **8** |  B   |
| _F_    | _9_ |  D   |

```
unvisited = [C,F]
```

### Next Node: F

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| A    |  0   |     |
| B    | 2 |  A    |
| _C_    | _17_ |  E   |
| D   | 7 |   B  |
| E    | 8 |  B   |
| **F**    | **9** |  D   |

```
unvisited = [C,F]
```

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| A    |  0   |     |
| B    | 2 |  A    |
| _C_    | _9+3 (<17)_ |  E   |
| D   | 7 |   B  |
| E    | 8 |  B   |
| **F**    | **9** |  D   |

```
unvisited = [C,F]
```

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| A    |  0   |     |
| B    | 2 |  A    |
| _C_    | _12_ |  F   |
| D   | 7 |   B  |
| E    | 8 |  B   |
| **F**    | **9** |  D   |

```
unvisited = [C]
```

### Next Node: C

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| A    |  0   |     |
| B    | 2 |  A    |
| **C**    | **12** |  F   |
| D   | 7 |   B  |
| E    | 8 |  B   |
| F    | 9 |  D   |

```
unvisited = [C]
```

_No remaining unvisited neighbours_

| Node | Shortest Distance from A | Previous Node |
|------|--------------------------|---------------|
| A    |  0   |     |
| B    | 2 |  A    |
| **C**    | **12** |  F   |
| D   | 7 |   B  |
| E    | 8 |  B   |
| F    | 9 |  D   |

```
unvisited = []
```

The length of the shortest path between $A$ and $C$ is 12. But we want to find _what that path is_.

## Shortest Path

Luckily, we have this information in our list of previous nodes, `prev` in the previous psuedocode.

We can back track from the target until we get to the initial node, as for each node, the previous node is the one from which it has shortest path to the initial. We can iterate through by creating a stack, pushing the target node first. The next node to be pushed is its previous node. Then we push the previous node of _that_ node, and so on until we have pushed the $A$ to the stack. The stack then represents the shortest path from the initial to the target.

```python
def shortest_path(init, target, prev)
    u = target
    path = stack()
    path.push(u)
    while u is not init:
        u = prev[u]
        path.push(u)
    return path
```

C

F

D

B

A

### Shortest Path Agent

We can implement this using an agent-based approach, by defining our environment as a graph with nodes (more details in Lab 3).

We have a set of graph, made up of nodes which have a set of neighbours defined by tuples `(node, weight)`.

Our shortest path environment will provide the agent with its current location's neighbours as a percept (its location will always be a node on the graph).

We will consider the environment done when it has been provided with the shortest path, and it can execute two possible actions: the agent exploring a node, or delivering the shortest path to the target.

```python
class ShortestPathEnvironment(GraphEnvironment):

    def percept(self, agent):
        return agent.location.neighbours
    
    def add_agent(self, agent, init, target):
        agent.initialise(init, target)
        self.agents.add(agent)

    @property
    def is_done(self):
        return hasattr(self, "shortest_path")
    
    def execute_action(self, agent, action):
        command, node = action
        match command:
            case "explore":
                agent.explore(node)
            case "deliver":
                self.shortest_path = agent.deliver()
```

The agent is an example of a utility-based agent, as it is seeking to maximise its utility by choosing the _best_ choice for the next node to explore.

The agent will act by exploring nodes until it has reached its goal: i.e. it has visited the target node.

The utility of a given action is defined by the _negative_ distance of a possible node to the initial state. Maximising the negative distance is equivalent to choosing the smallest distance.

```python
class ShortestPathAgent(UtilityBasedAgent):
    def __init__(self):
        self.dist = {}
        self.prev = {}
        self.visited = set()

    @property
    def at_goal(self):
        return self.location is self.target

    def initialise(self, node, target):
        self.init = node
        self.target = target
        self.dist[node] = 0
    
    def explore(self, node):  # actuator
        self.visited.add(self.location)
        self.location = node

    def deliver(self):  # actuator
        path = stack()
        curr = self.target
        path.push(curr)
        while curr is not self.init:
            curr = self.prev[curr]
            path.push(curr)
        return path, self.dist[node]

    
    def program(self, percepts):
        if self.at_goal:
            return ("deliver", self.target)
            
        for neighbour, weight in percepts:
            if neighbour in self.visited:
                continue
                
            alt = self.dist[self.location] + weight
            if neighbour not in self.dist or  # equivalent to dist[neighbour] = inf
                    alt < self.dist[neighbour]:
                self.dist[neighbour] = alt
                self.prev[neighbour] = self.location
                    
        action = self.maximise_utility(
            [("explore", node) 
                 for node in self.dist if (
                     node is not self.location and
                         node not in self.visited)])
        
        return action

    def utility(self, action):
        value = self.dist[action[1]]
        return -value  # maximising a value is equivalent to minimising its negative
```

We can see this in action on the graph, and see that our example above is correct:

In [ ]:
environment = ShortestPathEnvironment.from_dict(graph)

environment.add_agent(ShortestPathAgent(), init="A", target="C")

environment.show()
environment.run(pause_for_user=False)

## Exercise: Shortest Path between UK Cities

Try out the `ShortestPathEnvironment` and `Agent` with the UK cities network from the lecture:

![](https://raw.githubusercontent.com/wilocw/co2114-codebase/2024/static/0/UK_search_nodes.png)

In [ ]:
network = UK_CITIES_GRAPH
display(list(zip(network['edges'], network['weights'])))

In [ ]:
environment = ShortestPathEnvironment.from_dict(network)

environment.add_agent(ShortestPathAgent(), init="Sheffield", target="Cardiff")
environment.run(pause_for_user=False)